In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd #data-processing
import seaborn as sns # for plots 
import matplotlib.pyplot as plt # for visualizations and plots
import plotly.express as px #for plots
import numpy as np

In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
df.head()

# Getting Info about the dataset

In [ ]:
df.info()


In [ ]:
df.describe()

# Checking NaNs

In [ ]:
df.isna().sum().sort_values(ascending=True)

# EDA

In [ ]:

plt.figure(figsize=(20,6))
sns.lineplot(data=df)

In [ ]:
#outcome distrbution

sns.displot(data=df,x='Outcome')

In [ ]:
px.pie(df, names="Outcome")


****

**BloodPressure VS Outcome**

In [ ]:
sns.countplot(data=df,x='BloodPressure',hue='Outcome',palette='Set3')

In [ ]:
sns.displot(data=df,x='BloodPressure',hue='Outcome',kde=True)

*Blood preesure for diapetics are high*

**Glucose VS Outcome**

In [ ]:
sns.displot(data=df,x='Glucose',hue='Outcome',kde=True)

*Glucose of diapetics are also high*

****

**Pregnencies vs Outcome**

In [ ]:
sns.displot(data=df,x='Pregnancies',hue='Outcome',kde=True)

In [ ]:
sns.countplot(data=df,x='Pregnancies',hue='Outcome')

**SkinThickness VS Outcome**

In [ ]:
sns.histplot(data=df,x='SkinThickness',hue='Outcome',kde=True)

****

**Insulin vs Outcome**

In [ ]:
sns.histplot(data=df,x='Insulin',hue='Outcome',kde=True)

**age VS outcome**

In [ ]:
sns.histplot(data=df,x='Age',hue='Outcome',kde=True)

**BMI VS outcome**

In [ ]:
sns.histplot(data=df,x='BMI',hue='Outcome',kde=True)

****

**DiabetesPedigreeFunction VS outcome**

In [ ]:
sns.histplot(x="DiabetesPedigreeFunction", hue="Outcome", data=df, kde=True)

**Pairplot**

In [ ]:
sns.pairplot(df, hue='Outcome')

**Boxplot**

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(20,20))
axs = axs.flatten()
for i in range(len(df.columns)-1):
    sns.boxplot(data=df, x=df.columns[i], ax=axs[i])

**Correlation matrix**

In [ ]:
plt.subplots(figsize=(20,15))

sns.heatmap(df.corr(), linewidths=0.1, vmax=1.0, square=True, cmap='coolwarm', linecolor='white', annot=True).set_title("Correlation Map")

**Check Skewness**

In [ ]:
df["Glucose"] = df["Glucose"].apply(lambda x: np.nan if x == 0 else x)
df["BloodPressure"] = df["BloodPressure"].apply(lambda x: np.nan if x == 0 else x)
df["SkinThickness"] = df["SkinThickness"].apply(lambda x: np.nan if x == 0 else x)
df["Insulin"] = df["Insulin"].apply(lambda x: np.nan if x == 0 else x)
df["BMI"] = df["BMI"].apply(lambda x: np.nan if x == 0 else x)

In [ ]:
df.skew()


impute highly skewed values with median, else mean 

In [ ]:
df['Insulin'].isna().sum()

In [ ]:
df.drop(columns=["Insulin"], inplace=True)


In [ ]:
# Highly skewed

df["BMI"].replace(to_replace=np.nan,value=df["BMI"].median(), inplace=True)
df["Pregnancies"].replace(to_replace=np.nan,value=df["Pregnancies"].median(), inplace=True)
#df["Insulin"].replace(to_replace=np.nan,value=df["Insulin"].median(), inplace=True)

# Normal
df["Glucose"].replace(to_replace=np.nan,value=df["Glucose"].mean(), inplace=True)
df["BloodPressure"].replace(to_replace=np.nan,value=df["BloodPressure"].mean(), inplace=True)
df["SkinThickness"].replace(to_replace=np.nan,value=df["SkinThickness"].mean(), inplace=True)

# Dealing with Outliers

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
Tot_IQR = Q3 - Q1
print(Tot_IQR)

In [ ]:
df_out = df[~((df < (Q1 - 1.5 * Tot_IQR)) | (df > (Q3 + 1.5 * Tot_IQR))).any(axis=1)]
print(f'Before: {df.shape}, After: {df_out.shape}')

In [ ]:
df_out.describe()

# Splitting

In [ ]:
from sklearn.model_selection import train_test_split # spliting training and testing data
from sklearn.preprocessing import MinMaxScaler # data normalization with sklearn
from sklearn.preprocessing import StandardScaler # data standardization with  sklearn
from sklearn.ensemble import RandomForestClassifier # model
from sklearn.linear_model import LogisticRegression # model
from sklearn.neighbors import KNeighborsClassifier # model
from sklearn.metrics import classification_report, confusion_matrix # to evaluate the model
from mlxtend.plotting import plot_confusion_matrix # plot confusion matrix
from sklearn.model_selection import GridSearchCV # to finetune the model

In [ ]:
X = df_out[df_out.columns[:-1]]
y = df_out['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# Normalize

In [ ]:
norm = MinMaxScaler().fit(X_train)
X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)

# Models

In [ ]:
log_params = {'C': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 100, 100]} 
log_model = GridSearchCV(LogisticRegression(), log_params, cv=5)
log_model.fit(X_train_norm, y_train)
log_pred = log_model.predict(X_test_norm)

In [ ]:
rf_params = {'criterion' : ['gini', 'entropy'],
             'n_estimators': list(range(60, 140, 20)),
             'max_depth': list(range(3, 20, 2))}
rf_model = GridSearchCV(RandomForestClassifier(), rf_params, cv=5)
rf_model.fit(X_train_norm, y_train)
rf_pred = rf_model.predict(X_test_norm)

In [ ]:
knn_params = {'n_neighbors': list(range(1,50))}
knn_model = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5)
knn_model.fit(X_train_norm, y_train)
knn_pred = knn_model.predict(X_test_norm)

In [ ]:
from xgboost import XGBClassifier


XGB_params = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]}
XGB_model = GridSearchCV(XGBClassifier(),XGB_params,cv=5)
XGB_model.fit(X_train_norm,y_train)
XGB_pred = XGB_model.predict(X_test_norm)

# Evaluation

In [ ]:
print("Logistic Regression: \n", classification_report(y_test, log_pred)) 
print("\nRandom Forest Classifier: \n", classification_report(y_test, rf_pred)) 
print("\nK Neighbors Classifier: \n", classification_report(y_test, knn_pred))
print("\n XGB Calssifier: \n", classification_report(y_test, XGB_pred))


> Random calssifier has the best recall and precision

# Confusion Matrix

In [ ]:
labels = ["Not Diabetic", "Diabetic"]
cm  = confusion_matrix(y_test, rf_pred)
plt.figure()
plot_confusion_matrix(cm, hide_ticks=True, cmap="Blues")
plt.xticks(range(2), labels, fontsize=14)
plt.yticks(range(2), labels, fontsize=14)
plt.show()

In [ ]:
labels = ["Not Diabetic", "Diabetic"]
cm  = confusion_matrix(y_test, XGB_pred)
plt.figure()
plot_confusion_matrix(cm, hide_ticks=True, cmap="Blues")
plt.xticks(range(2), labels, fontsize=14)
plt.yticks(range(2), labels, fontsize=14)
plt.show()

# Calculte roc_auc Score

In [ ]:
from sklearn.metrics import roc_auc_score

# auc scores
auc_score1 = roc_auc_score(y_test, rf_pred)
auc_score2 = roc_auc_score(y_test, knn_pred)
auc_score3 = roc_auc_score(y_test, log_pred)
auc_score4 = roc_auc_score(y_test,XGB_pred)

print(auc_score1,auc_score2,auc_score3, auc_score4)

